<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/main/notebooks/Federated_learning_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from torchvision.datasets import CIFAR100
from torch.utils.data import Subset
import numpy as np
import copy
import random
import torch
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:06<00:00, 26.7MB/s]


In [ ]:
clients_data = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")
clients_data.head()

,client_id,backbone,model_name,initial_model_name,path,num_of_clients,Measurement_criteria,accuracy,loss,train_loss,...,size_of_dataset,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs
0,0,dino_vits16,28b80fff-61ff-445b-8c19-6210b0fefd2e,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.960880,0.162077,0.218887,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,66.262507,2025-05-27 09:34:10,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
1,1,dino_vits16,66ce83c7-9027-469e-b937-4040d8ef3d02,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",97.555012,0.157862,0.183864,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,65.349265,2025-05-27 09:35:23,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
2,8,dino_vits16,34fe6e15-49ea-4530-a9c2-91fb18b9c71a,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.471883,0.179580,0.226160,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,67.385976,2025-05-27 10:15:59,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
3,14,dino_vits16,e00bd69c-dade-4936-bbd5-beb9f9138bd0,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.349633,0.174917,0.211031,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,62.720521,2025-05-27 10:17:15,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
4,9,dino_vits16,a026faa9-5fa9-4da4-9455-c5fbcd0ade4e,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.410758,0.177857,0.207068,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,63.000698,2025-05-27 10:23:15,/content/drive/MyDrive/MLDL_FederatedLearning/...,/content/drive/MyDrive/MLDL_FederatedLearning/...


In [ ]:
initial_model_name = "4f20c1df-2f32-48b8-8289-3acac23e314f" #@param{"type":"string"}

In [ ]:

filter =  clients_data['initial_model_name']== initial_model_name
filtered_clients_data = clients_data[filter]
params = filtered_clients_data[['backbone',
                                    'num_of_clients',
                                    'splitting_method',
                                    'size_of_dataset']]
params = dict(params.iloc[0])
print("Number of all trained clients:", len(clients_data))
print("Number of clients after filtering:", len(filtered_clients_data))
contributors = []
for i  in filtered_clients_data['model_name'].values:
  contributors.append(i)
print("Contributors:", contributors)
filtered_clients_data.head()

Number of all trained clients: 20
Number of clients after filtering: 20
Contributors: ['28b80fff-61ff-445b-8c19-6210b0fefd2e', '66ce83c7-9027-469e-b937-4040d8ef3d02', '34fe6e15-49ea-4530-a9c2-91fb18b9c71a', 'e00bd69c-dade-4936-bbd5-beb9f9138bd0', 'a026faa9-5fa9-4da4-9455-c5fbcd0ade4e', '0bd88bc1-87ac-4698-8123-e27b514b36e9', '6aaebdb6-faba-47e7-97d2-d6641c9ada2e', '53aae886-88c4-46d9-8ad0-e8b8a3c61321', '8d00792b-ee60-4194-a26f-63f816e78999', '53668afe-0f6b-49ce-8cb7-a5a9a41ea900', '788b130e-335b-4819-933b-5f324859654e', 'e85dcb66-3ac6-4a1e-abbc-d820a77c421a', '4d652197-b82e-4aa0-b037-11ec66fadc4e', 'e5392c98-735b-43cc-9b6e-9277eb2027ff', 'adf93988-63d3-41a8-9f27-6098680e2e8d', '7addc050-1c52-41bd-8ce7-f6e93a4487f3', 'c437e824-48e4-40f8-b1b2-60cfdcccbbe5', '16c17850-8025-486c-8380-d741e5c0dc78', '0f3c6bf7-1129-456a-86fb-5d83ab72a676', '9fb60fee-69bf-4328-898d-37b89332f799']


,client_id,backbone,model_name,initial_model_name,path,num_of_clients,Measurement_criteria,accuracy,loss,train_loss,...,size_of_dataset,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs
0,0,dino_vits16,28b80fff-61ff-445b-8c19-6210b0fefd2e,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.960880,0.162077,0.218887,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,66.262507,2025-05-27 09:34:10,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
1,1,dino_vits16,66ce83c7-9027-469e-b937-4040d8ef3d02,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",97.555012,0.157862,0.183864,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,65.349265,2025-05-27 09:35:23,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
2,8,dino_vits16,34fe6e15-49ea-4530-a9c2-91fb18b9c71a,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.471883,0.179580,0.226160,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,67.385976,2025-05-27 10:15:59,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
3,14,dino_vits16,e00bd69c-dade-4936-bbd5-beb9f9138bd0,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.349633,0.174917,0.211031,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,62.720521,2025-05-27 10:17:15,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
4,9,dino_vits16,a026faa9-5fa9-4da4-9455-c5fbcd0ade4e,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.410758,0.177857,0.207068,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,63.000698,2025-05-27 10:23:15,/content/drive/MyDrive/MLDL_FederatedLearning/...,/content/drive/MyDrive/MLDL_FederatedLearning/...


In [ ]:
#@title Dino Model
import torch
import torch.nn as nn

class DinoClassifire(nn.Module):
  def __init__(self, backbone, num_classes:int=100, device=None):
    super(DinoClassifire, self).__init__()
    self.backbone = torch.hub.load('facebookresearch/dino:main', backbone)

    #We need to freaze thhe parameters of bakbone first so we can train only on the head layer(output layer)
    for param in self.backbone.parameters():
      param.requires_grad = False

    #determine the Device
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.backbone.to(device)

    #To detect the output feature dimontion of backbone we run  Dummy forward pass
    with torch.no_grad():

      dummy_input = torch.randn(1,3,224,224).to(device)
      dummy_out = self.backbone(dummy_input)


      if isinstance(dummy_out, tuple):
        dummy_out = dummy_out[0]
      elif isinstance(dummy_out, dict):
        dummy_out = dummy_out.get("x_norm_clstoken", next(iter(dummy_out.values())))

      #If the output is 3D (B, T, D), we assume first token is the [CLS] token.
      if dummy_out.dim() == 3:
        dummy_feature = dummy_out[:,0]
      else:
        dummy_feature = dummy_out
      feature_dim = dummy_feature.shape[1]
      print("Detected feature dimontion:", feature_dim)


      #Difineing the classification Head
      self.head = nn.Linear(feature_dim, num_classes)

      #Ensure the head is trainable.
      for param in self.head.parameters():
        param.requires_grad = True

  def forward(self,x):

    #pass the input through the backbone
    features = self.backbone(x)

    if isinstance(features, tuple):
      features = features[0]
    elif isinstance(features, dict):
      features = features.get("x_norm_clstoken", next(iter(features.values())))


    # If featers are retuened as (B, T, D), use the first token
    if features.dim() == 3:
      cls_token = features[:,0]
    else:
      cls_token = features
    logits = self.head(cls_token)

    return logits

In [ ]:
def get_model(paths,sample_sizes, backbone):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = DinoClassifire(backbone=backbone, num_classes=100, device=device)
  for index in range(len(paths)):
    state_dict = torch.load(paths.iloc[index])
    model.load_state_dict(state_dict)
    model.to(device)
    yield (model,sample_sizes.iloc[index])

In [ ]:
models = list(get_model(filtered_clients_data["path"],filtered_clients_data["client_train_size"], params["backbone"]))

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall16_pretrain.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 169MB/s]


Detected feature dimontion: 384


In [ ]:
#@title select aggregation method

aggregation_method = "FebAvg" #@param["FebAvg","FebAvgM", "EMA" ]


In [ ]:

def feb_avg(df):
  total_samples = df["client_train_size"].sum()
  global_head = None

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if global_head is None:
        global_head = copy.deepcopy(model.head.state_dict())
        global_model = copy.deepcopy(model)
        for k in global_head.keys():
          global_head[k].zero_()

      for k in global_head.keys():
        global_head[k] += model.head.state_dict()[k] * (sample_size / total_samples)

    global_model.head.load_state_dict(global_head)

  return global_model



In [ ]:
def feb_avg_m(df, momentum_coefficient=0.9, cliping_range=5,  momentum_vector_path= None):

  total_samples = df["client_train_size"].sum()
  delta = None

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if delta is None:

        global_model = copy.deepcopy(model)
        global_head = copy.deepcopy(model.head.state_dict())
        delta = { k: torch.zeros_like(v) for k, v in global_head.items() }


      client_head = model.head.state_dict()
      for k in delta.keys():

        delta[k] += (client_head[k] - global_head[k]) * (sample_size / total_samples)

    if momentum_vector_path is None :
      momentum_vector = copy.deepcopy(delta)
      for k in delta.keys():
        momentum_vector[k] = torch.clamp(delta[k], -cliping_range, cliping_range)
    else:
      momentum_vector = torch.load(momentum_vector_path)
      for k in delta.keys():
        momentum_vector[k] = momentum_coefficient * momentum_vector[k]  + delta[k]


    global_head = copy.deepcopy(global_model.head.state_dict())
    for k in momentum_vector.keys():
      global_head[k] = torch.clamp(global_head[k] + momentum_vector[k], -cliping_range, cliping_range)

    global_model.head.load_state_dict(global_head)

  return global_model, momentum_vector



In [2]:
def ema(df, decay=0.9, cliping_range=5, momentum_vector_path= None):

  total_samples = df["client_train_size"].sum()
  delta = None

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if delta is None:

        global_model = copy.deepcopy(model)
        global_head = copy.deepcopy(model.head.state_dict())
        delta = { k: torch.zeros_like(v) for k, v in global_head.items() }


      client_head = model.head.state_dict()
      for k in delta.keys():

        delta[k] += (client_head[k] - global_head[k]) * (sample_size / total_samples)

    if momentum_vector_path is None :
      momentum_vector = copy.deepcopy(delta)
    else:
      momentum_vector = torch.load(momentum_vector_path)
      for k in delta.keys():
        momentum_vector[k] = decay * momentum_vector[k]  + (1- decay) * delta[k]


    global_head = copy.deepcopy(global_model.head.state_dict())
    for k in momentum_vector.keys():
      global_head[k] = torch.clamp(global_head[k] + momentum_vector[k], -cliping_range, cliping_range)

    global_model.head.load_state_dict(global_head)

  return global_model, momentum_vector



In [3]:
def eveluation(model, data_loader):
  criterion = nn.CrossEntropyLoss()
  model.eval()
  correct = 0
  total = 0
  test_loss = 0

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  with torch.no_grad():
    for images, labels in  data_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      _, prediction = torch.max(outputs.data,1)
      loss = criterion(outputs, labels)
      test_loss += loss.item() * labels.size(0)

      total += labels.size(0)
      correct += (prediction == labels).sum().item()
    accuracy = 100 * correct / total
    loss = test_loss / total
    return accuracy, loss



In [ ]:
from uuid import uuid4
import os
def next_id(log_path):
  if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    while True:
      uuid = str(uuid4())
      if uuid not in df["model_name"].values:
        return uuid
  else:
    return str(uuid4())

In [1]:
from datetime import datetime

def get_current_time():
  now = datetime.now()

  formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S") # Format the date and time as a string

  return formatted_date_time

def global_model_name_path_generator(record):
  method = "iid" if record["splitting_method"] == "i.i.d. sharing" else "noniid"

  model_name = next_id("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")

  path = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/" + model_name

  return model_name, path



In [ ]:
def del_model(model_name):
  log_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  filter = log_df["model_name"] == model_name
  if not filter.any():
    print(f"Model ({model_name}) not found.")
    return

  os.remove(log_df[filter]["path"].values[0])
  if log_df[filter]["momentum_vector_path"].values[0] is not None:
    os.remove(log_df[filter]["momentum_vector_path"].values[0])
  log_df = log_df[~filter]
  log_df.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", index=False)


In [ ]:

if aggregation_method == "FebAvg":
  global_model = feb_avg(filtered_clients_data)
elif aggregation_method == "FebAvgM":
  global_model,momentum_vector = feb_avg_m(filtered_clients_data)
elif aggregation_method == "EMA":
  global_model,momentum_vector = ema(filtered_clients_data)





test_accracy, test_loss= eveluation(global_model, test_loader)
print(f"Accurace:{test_accracy:.2f}")
print(f"Loss:{test_loss:.2f}")

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


Detected feature dimontion: 384


In [ ]:
import os
log_path = "/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv"

model_name, path = global_model_name_path_generator(dict(filtered_clients_data.iloc[0]))
prev_global_model_name = filtered_clients_data["initial_model_name"].values[0]
global_model_log = filtered_clients_data.drop(["client_id","train_loss","client_train_size","client_test_size","duration",],axis=1)
global_model_log = global_model_log.iloc[0]
global_model_log["num_of_participants"] = len(filtered_clients_data)
global_model_log["prev_global_model_name"] = initial_model_name
global_model_log["model_name"]= model_name
global_model_log["accuracy"] = test_accracy
global_model_log["loss"] = test_loss
global_model_log["time"] = get_current_time()
global_model_log["path"] = path
global_model_log["contributors"] = contributors

if aggregation_method == "FebAvg":
  global_model_log["momentum_vector_path"] = None


elif aggregation_method == "FebAvgM":
  torch.save(momentum_vector, global_model_log["momentum_vector_path"])
  global_model_log["momentum_vector_path"] = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/momentun_vectors/MV_"+ model_name + ".pt"



global_model_log = pd.DataFrame(global_model_log).T
global_model_log = global_model_log[['backbone', 'model_name', 'num_of_clients', 'path',
       'Measurement_criteria', 'prev_global_model_name', 'accuracy', 'loss',
       'splitting_method', 'size_of_dataset', 'train_test_ratio', 'classes',
       'round_number', 'time', 'path_to_subsets', 'path_to_class_combs',
       'num_of_participants',"contributors"]]


flag = False
if os.path.exists(path):
  print("Model already exist")
  flag = True

if os.path.exists(log_path):
  global_log_df = pd.read_csv(log_path)
  if model_name in global_log_df["model_name"].values:
    print("record already exist")
    flag = True
  if not flag:
    global_model_log.to_csv(log_path, mode='a', header=False, index=False)
    print("existing log file")
else:
  if not flag:
    global_model_log.to_csv(log_path, index=False, header=True)
    print("new log file")



if not flag:
  torch.save(global_model.state_dict(),  path )

In [ ]:
global_model_log.head()